<a href="https://colab.research.google.com/github/joao-fcosta/API_IA/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import pandas as pd

#from google.colab import drive

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [40]:
# =============================
# 2. Carregar Dataset
# =============================
#drive.mount('/content/drive')
#df = pd.read_csv('/content/drive/MyDrive/dataset/API_IA/FuelConsumptionCo2.csv')
df = pd.read_csv('./data/Fuel_Consumption.csv')
df.head()


,YEAR,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,FUEL CONSUMPTION,HWY (L/100 km),COMB (L/100 km),COMB (mpg),EMISSIONS
0,2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,9.2,6.7,8.1,35,186
1,2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,8.5,6.5,7.6,37,175
2,2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,12.2,7.4,10.0,28,230
3,2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,13.4,9.2,11.5,25,264
4,2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,10.0,7.0,8.6,33,198


In [41]:
# 2) SELECIONAR FEATURES E ALVO
features = ["YEAR", "MAKE","MODEL","ENGINE SIZE", "CYLINDERS", "VEHICLE CLASS", "FUEL"]
target = "FUEL CONSUMPTION"

data = df[features + [target]].dropna()

X = data[features]
y = data[target]

X.head(), y.head()


(   YEAR   MAKE    MODEL  ENGINE SIZE  CYLINDERS VEHICLE CLASS FUEL
 0  2000  ACURA    1.6EL          1.6          4       COMPACT    X
 1  2000  ACURA    1.6EL          1.6          4       COMPACT    X
 2  2000  ACURA    3.2TL          3.2          6      MID-SIZE    Z
 3  2000  ACURA    3.5RL          3.5          6      MID-SIZE    Z
 4  2000  ACURA  INTEGRA          1.8          4    SUBCOMPACT    X,
 0     9.2
 1     8.5
 2    12.2
 3    13.4
 4    10.0
 Name: FUEL CONSUMPTION, dtype: float64)

In [42]:
# 3) TREINO / TESTE
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [43]:
# 4) PRÉ-PROCESSAMENTO

numeric_features = ["ENGINE SIZE", "CYLINDERS", "YEAR"]
categorical_features = ["VEHICLE CLASS", "MAKE","MODEL", "FUEL"]

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)



In [44]:
# 5) DEFINIR MODELOS

ridge_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", Ridge())
])

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])

gb_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", GradientBoostingRegressor(random_state=42))
])


In [45]:
# 4) GRIDS DE HIPERPARÂMETROS (VERSÃO ENXUTA)

param_grid_ridge = {
    "model__alpha": [0.1, 1.0, 10.0]
}

param_grid_rf = {
    "model__n_estimators": [300],        # fixo
    "model__max_depth": [None, 10],      # 2 opções
    "model__min_samples_split": [2, 5]   # 2 opções
}

param_grid_gb = {
    "model__n_estimators": [200],        # fixo
    "model__learning_rate": [0.05, 0.1], # 2 opções
    "model__max_depth": [2, 3]           # 2 opções
}


''' param_grid_ridge = {
    "model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]
}

param_grid_rf = {
    "model__n_estimators": [200, 400, 600],
    "model__max_depth": [None, 8, 12],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2"]
}

param_grid_gb = {
    "model__n_estimators": [100, 300, 500],
    "model__learning_rate": [0.03, 0.05, 0.1],
    "model__max_depth": [2, 3, 4],
    "model__subsample": [0.8, 1.0],
    "model__min_samples_leaf": [1, 3, 5]
} '''


' param_grid_ridge = {\n    "model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]\n}\n\nparam_grid_rf = {\n    "model__n_estimators": [200, 400, 600],\n    "model__max_depth": [None, 8, 12],\n    "model__min_samples_split": [2, 5, 10],\n    "model__min_samples_leaf": [1, 2, 4],\n    "model__max_features": ["sqrt", "log2"]\n}\n\nparam_grid_gb = {\n    "model__n_estimators": [100, 300, 500],\n    "model__learning_rate": [0.03, 0.05, 0.1],\n    "model__max_depth": [2, 3, 4],\n    "model__subsample": [0.8, 1.0],\n    "model__min_samples_leaf": [1, 3, 5]\n} '

In [46]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)

# RIDGE
grid_ridge = GridSearchCV(
    estimator=ridge_pipeline,
    param_grid=param_grid_ridge,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# RANDOM FOREST
grid_rf = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid_rf,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# GRADIENT BOOSTING
grid_gb = GridSearchCV(
    estimator=gb_pipeline,
    param_grid=param_grid_gb,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid_ridge.fit(X_train, y_train)
grid_rf.fit(X_train, y_train)
grid_gb.fit(X_train, y_train)

print("Ridge - melhor alpha:", grid_ridge.best_params_, "RMSE (CV):", -grid_ridge.best_score_)
print("RF    - best params:",  grid_rf.best_params_,    "RMSE (CV):", -grid_rf.best_score_)
print("GB    - best params:",  grid_gb.best_params_,    "RMSE (CV):", -grid_gb.best_score_)


KeyboardInterrupt: 

In [ ]:
best_ridge = grid_ridge.best_estimator_
best_rf    = grid_rf.best_estimator_
best_gb    = grid_gb.best_estimator_

for nome, modelo in [("Ridge", best_ridge),
                     ("RandomForest", best_rf),
                     ("GradientBoosting", best_gb)]:
    y_pred = modelo.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2   = r2_score(y_test, y_pred)
    print(f"{nome} -> RMSE teste: {rmse:.3f} L/100km | R²: {r2:.3f}")


Ridge -> RMSE teste: 0.882 L/100km | R²: 0.938
RandomForest -> RMSE teste: 0.906 L/100km | R²: 0.934
GradientBoosting -> RMSE teste: 0.996 L/100km | R²: 0.921


In [ ]:
# 10) ENSEMBLE COM VOTINGREGRESSOR

voting_reg = VotingRegressor(
    estimators=[
        ("ridge", best_ridge),
        ("rf",    best_rf),
        ("gb",    best_gb)
    ]
)

# Cross-val com RMSE
scores = cross_val_score(
    voting_reg,
    X, y,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

print("VotingRegressor - RMSE médio (CV):", -np.mean(scores))
print("VotingRegressor - RMSE desvio padrão:", np.std(-scores))

# Avaliação no teste
voting_reg.fit(X_train, y_train)
y_pred_voting = voting_reg.predict(X_test)
rmse_voting = np.sqrt(mean_squared_error(y_test, y_pred_voting))
r2_voting   = r2_score(y_test, y_pred_voting)

print("VotingRegressor - RMSE teste:", rmse_voting)
print("VotingRegressor - R² teste:", r2_voting)


VotingRegressor - RMSE médio (CV): 0.8700668031258557
VotingRegressor - RMSE desvio padrão: 0.032155226994638325
VotingRegressor - RMSE teste: 0.8693606892273936
VotingRegressor - R² teste: 0.9394991443456596


In [ ]:
import joblib

joblib.dump(voting_reg, "modelo_combustivel.joblib")
print("Modelo salvo em modelo_combustivel.joblib")


Modelo salvo em modelo_combustivel.joblib
